In [1]:
import pandas as pd
import numpy as np
import re
import pandas as pd

import matcher  
import spacy


In [67]:
# Cargar los datos del archivo CSV
df = matcher.get_df('colores.csv')

In [69]:
# nlp = spacy.load("en_core_web_sm")
nlp = spacy.load("fr_core_news_sm")

In [47]:
nlp = spacy.load("es_core_news_sm")

In [ ]:


# Crear un DataFrame de ejemplo
data = {
    "texto": ["Esto es una prueba de lematización.", "Los gatos están durmiendo."],
}
df = pd.DataFrame(data)

# Definir una función para lematizar el texto en la columna


# Aplicar la función a la columna de texto
df["lemmatized_text"] = df["texto"].apply(lemmatize_text)

# Mostrar el DataFrame resultante
print(df)

In [15]:
def lemmatize_text(text):
    doc = nlp(text)
    lemmatized_text = " ".join([token.lemma_ for token in doc])
    return lemmatized_text

In [ ]:
lemmatized_text = " ".join([token.lemma_ for token in df])

In [ ]:
doc = nlp("naranja")
for token in doc:
    print(token.text, token.lemma_)

In [71]:
df["word"] = df["word"].map(lambda x: x.lower())
df["lemmatized_text"] = df["word"].apply(lemmatize_text)
df["tag"] = "color"
df["language"] = "fr"

In [72]:
df

,word,lemmatized_text,tag,language
0,rouge,rouge,color,fr
1,bleu,bleu,color,fr
2,jaune,jaune,color,fr
3,orange,oranger,color,fr
4,vert,vert,color,fr
5,violet,violet,color,fr
6,rouge√¢tre,rouge√¢tr,color,fr
7,vert olive,vert olive,color,fr
8,jeanne citron,jeanne citron,color,fr
9,blanc,blanc,color,fr


In [42]:
from sqlalchemy import create_engine


In [73]:
# Establecer una conexión con la base de datos SQLite (si no existe, la creará)
engine = create_engine('sqlite:///data.sqlite', echo=True)

# Guardar el DataFrame en la base de datos (reemplazando si ya existe)
df.to_sql('LemmatizedKeywords', con=engine, if_exists='append', index=False)



2024-04-15 20:31:19,034 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-04-15 20:31:19,037 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("LemmatizedKeywords")
2024-04-15 20:31:19,038 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-04-15 20:31:19,042 INFO sqlalchemy.engine.Engine INSERT INTO "LemmatizedKeywords" (word, lemmatized_text, tag, language) VALUES (?, ?, ?, ?)
2024-04-15 20:31:19,044 INFO sqlalchemy.engine.Engine [generated in 0.00161s] [('rouge', 'rouge', 'color', 'fr'), ('bleu', 'bleu', 'color', 'fr'), ('jaune', 'jaune', 'color', 'fr'), ('orange', 'oranger', 'color', 'fr'), ('vert', 'vert', 'color', 'fr'), ('violet', 'violet', 'color', 'fr'), ('rouge√¢tre', 'rouge√¢tr', 'color', 'fr'), ('vert olive', 'vert olive', 'color', 'fr')  ... displaying 10 of 12 total bound parameter sets ...  ('noir', 'noir', 'color', 'fr'), ('gris', 'gris', 'color', 'fr')]
2024-04-15 20:31:19,063 INFO sqlalchemy.engine.Engine SELECT name FROM sqlite_master WHERE type='table' AND name NOT

12

In [65]:
# Consultar la tabla para verificar si se ha cargado correctamente
consulta = pd.read_sql_query('SELECT * FROM LemmatizedKeywords', engine)
print(consulta)

2024-04-15 16:55:42,422 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-04-15 16:55:42,424 INFO sqlalchemy.engine.Engine SELECT * FROM LemmatizedKeywords
2024-04-15 16:55:42,424 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-04-15 16:55:42,449 INFO sqlalchemy.engine.Engine ROLLBACK
          word lemmatized_text    tag language
0         rojo            rojo  color       es
1         azul            azul  color       es
2     amarillo        amarillo  color       es
3      naranja        naranjar  color       es
4        verde           verde  color       es
5      violeta        violetar  color       es
6   anaranjado      anaranjado  color       es
7       morado           morar  color       es
8   verde lima      verde lima  color       es
9       blanco          blanco  color       es
10       negro           negro  color       es
11        gris            gris  color       es
12         red             red  color       en
13        blue            blue  color       en
14     

In [66]:
consulta.to_csv("respaldo.csv")

In [36]:

# Process a text
doc = nlp("I am running in the park")

# Access lemmatized form of each token in the document
for token in doc:
    print(token.text, token.lemma_)

I I
am be
running run
in in
the the
park park


In [6]:
train_data

[[('Rojo', 'COLOR'),
  ('Azul', 'COLOR'),
  ('Amarillo', 'COLOR'),
  ('Naranja', 'COLOR'),
  ('Verde', 'COLOR'),
  ('Violeta', 'COLOR'),
  ('Anaranjado', 'COLOR'),
  ('Morado', 'COLOR'),
  ('Verde lima', 'COLOR'),
  ('Blanco', 'COLOR'),
  ('Negro', 'COLOR'),
  ('Gris', 'COLOR'),
  ('Red', 'COLOR'),
  ('Blue', 'COLOR'),
  ('Yellow', 'COLOR'),
  ('Orange', 'COLOR'),
  ('Green', 'COLOR'),
  ('Violet', 'COLOR'),
  ('Vermilion', 'COLOR'),
  ('Chartreuse', 'COLOR'),
  ('Magenta', 'COLOR'),
  ('White', 'COLOR'),
  ('Black', 'COLOR'),
  ('Gray', 'COLOR'),
  ('Rouge', 'COLOR'),
  ('Bleu', 'COLOR'),
  ('Jaune', 'COLOR'),
  ('Orange', 'COLOR'),
  ('Vert', 'COLOR'),
  ('Violet', 'COLOR'),
  ('Rougeâtre', 'COLOR'),
  ('Vert olive', 'COLOR'),
  ('Jeanne citron', 'COLOR'),
  ('Blanc', 'COLOR'),
  ('Noir', 'COLOR'),
  ('Gris', 'COLOR'),
  ('Красный (Krasniy)', 'COLOR'),
  ('Синий (Siniy)', 'COLOR'),
  ('Желтый (Zheltiy)', 'COLOR'),
  ('Оранжевый (Oranzhevyy)', 'COLOR'),
  ('Зеленый (Zeleniy)', 'COLOR'

In [13]:
matcher.detect_language_columns('files/farmatodo.json')

{'title': 'es', 'price': 'de', 'url': 'Unknown', 'input': 'es'}

In [7]:
lemmatized_colors = set(lemmatized_colors)


In [8]:
# Entrenar el etiquetador de unigramas
train_data = [[(color, 'COLOR') for color in lemmatized_colors]]
unigram_tagger = UnigramTagger(train_data)

In [14]:
unigram_tagger

<UnigramTagger: size=57>

In [9]:
def es_color(palabra):
    tagged_word = unigram_tagger.tag([palabra])
    if tagged_word[0][1] == 'COLOR':
        return True
    else:
        return False

In [11]:
es_color('rojizo')

False

In [22]:
# Función para verificar si una lista de palabras contiene colores
def contiene_colores(lista_palabras):
    return [palabra for palabra in lista_palabras if es_color(palabra)]

# Ejemplo de uso
lista_palabras = ['Rojo', 'Perro', 'Azul', 'Gato']
print(contiene_colores(lista_palabras)) # Output: ['Rojo', 'Azul']

['Rojo', 'Azul']


In [ ]:
# Convertir las etiquetas de categoría a números
y = y.map({cat: i for i, cat in enumerate(categorias)})

In [3]:
df = matcher.get_df('files/origin_Madison.csv')

In [8]:
df.apply(pd.unique(df))

C:\Users\Caritoshi\AppData\Local\Temp\ipykernel_13456\1888121351.py:1: FutureWarning: unique with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  df.apply(pd.unique(df))


ValueError: could not broadcast input array from shape (120,20) into shape (120,)